# Function Calling with Structured Output

### Install Required Libraries

In [17]:
!pip install -q openai
!pip install newsapi-python -q

### Upgrade Existing Installation

In [19]:
!pip install openai -U

  Attempting uninstall: openai
    Found existing installation: openai 1.42.0
    Uninstalling openai-1.42.0:
      Successfully uninstalled openai-1.42.0


### Load Environment Variables

In [21]:
import os
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv(), override=True)

os.environ.get('OPENAI_API_KEY')

print("API Key Loaded", os.environ.get('OPENAI_API_KEY') is not None)

API Key Loaded True


# News Articles Summarization

### Import Required Libraries

In [25]:
import openai
from openai import OpenAI
from pydantic import BaseModel, Field
import json
from newspaper import Article
from newsapi import NewsApiClient

### Create and Initialize Client

In [27]:
newsapi = NewsApiClient(api_key=os.environ.get('NEWS_API_KEY'))
client = OpenAI()

### Create Get News Without Structure

In [29]:
def get_news_summary_without_structure(query: str) -> str:
    """
    This function searches for a news article based on the query using News API,
    extracts the article content using the newspaper library, and summarizes it using OpenAI's GPT model 
    *without* enforcing strict adherence to a structured output schema.
    """

    top_headlines = newsapi.get_everything(q=query, 
                                          from_param='2024-08-01',
                                          to='2024-08-12',
                                          language='en',
                                          sort_by='relevancy',
                                          page=2)

    if top_headlines['status'] == 'ok' and top_headlines['articles']:
        article_url = top_headlines['articles'][0]['url']
        article = Article(article_url)
        article.download()
        article.parse()

        summary_prompt = f"Summarize the following news article in 2-3 sentences:\n\n{article.text}"
        summary_response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": "You are a helpful AI assistant that summarizes news articles."},
                {"role": "user", "content": summary_prompt}
            ]
        )
        summary = summary_response.choices[0].message.content

        return summary  

    else:
        print(top_headlines['status'])
        print(top_headlines['articles'])
        raise Exception("No news articles found for the given query.")

### Testing the function

In [32]:
get_news_summary_without_structure("Large Language Models")

"Elon Musk has filed a lawsuit against OpenAI and Sam Altman, accusing them of breaching contracts and violating federal laws against racketeering. Musk claims that Altman and OpenAI manipulated him into launching the artificial intelligence startup under false pretenses of neutrality and decentralization, allegedly planning to turn it into a for-profit entity despite being initially a non-profit. The lawsuit seeks damages, a constructive trust on assets traceable to Musk's contributions, and nullification of OpenAI's license to Microsoft."

### Define Pydantic Models

In [35]:
class NewsSummaryRequest(BaseModel):
    query: str = Field(..., description="The search query for news articles.")

class NewsSummaryResponse(BaseModel):
    summary: str = Field(..., description="A concise summary of the news article.")
    url: str = Field(..., description="The URL of the news article.")

### Fetch and Summarize News Articles Function with Structure Outputs

In [38]:
def get_news_summary(query: str) -> NewsSummaryResponse:
    """
    This function searches for a news article based on the query using News API,
    extracts the article content using the bbc-news library, and summarizes it using OpenAI's GPT model.
    """

    top_headlines = newsapi.get_everything(q='large language models',
                                      from_param='2024-08-01',
                                      to='2024-08-12',
                                      language='en',
                                      sort_by='relevancy',
                                      page=2)

    if top_headlines['status'] == 'ok' and top_headlines['articles']:
        article_url = top_headlines['articles'][0]['url']
        article = Article(article_url)
        article.download()
        article.parse()

        summary_prompt = f"Summarize the following news article in 2-3 sentences:\n\n{article.text}"
        summary_response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": "You are a helpful AI assistant that summarizes news articles."},
                {"role": "user", "content": summary_prompt}
            ]
        )
        summary = summary_response.choices[0].message.content

        return NewsSummaryResponse(summary=summary, url=article_url)
    else:
        print(top_headlines['status'])
        print(top_headlines['articles'])
        raise Exception("No news articles found for the given query.")

### Wrap the get_news_summary function in Pydantic Model

In [40]:
class NewsSummaryTool(BaseModel):
    def __call__(self, query: str) -> NewsSummaryResponse:
        return get_news_summary(query)

### Create the tool using openai.pydantic_function_tool with the wrapper model

In [42]:
tools = [openai.pydantic_function_tool(
    NewsSummaryTool, name="get_news_summary", description="Get a summary of a news article based on a query")
        ]

### Setup the messages array

In [44]:
messages = [
    {
        "role": "system", 
        "content": """
        You are a helpful assistant that can summarize news articles. Use the supplied tools to assist the user. 
        When using a tool, provide all the necessary arguments in a JSON format within the 'arguments' field of the tool call. 
        For example, to use the 'get_news_summary' tool, 
        provide the 'query' argument like this: {'query': 'latest news about Artificial Intelligence'}.
        """
    },
    {
        "role": "user", 
        "content": "Summarize the latest news about large language models"
    }
]

### Make the initial API call

In [46]:
response = client.chat.completions.create(
    model='gpt-4',  
    messages=messages,
    tools=tools,
    tool_choice={
        "type": "function", 
        "function": {
            "name": "get_news_summary" 
        },
        "strict": True   
    }
)

### Test the applicaiton (Check if the model wants to use a tool)

In [50]:
if response.choices[0].message.tool_calls:
    tool_call = response.choices[0].message.tool_calls[0]
    function_name = tool_call.function.name
    arguments = json.loads(tool_call.function.arguments)
    
    try:
        news_summary = get_news_summary(arguments['query'])
        
        function_call_result_message = {
            "role": "tool",
            "name": function_name,
            "content": news_summary.model_dump_json(), 
            "tool_call_id": tool_call.id
        }
        
        messages.append(response.choices[0].message)
        messages.append(function_call_result_message)
        
        final_response = client.chat.completions.create(
            model='gpt-4o-2024-08-06',  
            messages=messages
        )
        
        print(final_response.choices[0].message.content)

    except Exception as e:
        print(f"An error occurred: {str(e)}")
else:
    print(response.choices[0].message.content)

The latest news about large language models involves Elon Musk filing a lawsuit against OpenAI and Sam Altman. Musk accuses them of breaching contracts and engaging in racketeering. The lawsuit claims that Altman and OpenAI misled Musk into supporting the startup under the false guise of it being a neutral non-profit. Musk, who co-founded OpenAI in 2015 but left due to conflicts of interest, contributed over $44 million to the company. He seeks damages, a constructive trust on assets linked to his contributions, and a judicial declaration to nullify OpenAI's license agreements, specifically those concerning its large language models and Microsoft.

For more details, you can read the full article [here](https://qz.com/elon-musk-lawsuit-sam-altman-openai-tesla-microsoft-1851613031).
